In [21]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

import openai
import os
import dotenv

In [22]:
dotenv.load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [23]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

# LLMGraphTransformer를 사용하여 지식 그래프 추출
llm_transformer = LLMGraphTransformer(llm=llm)
text = """ Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted 
pioneering research on radioactivity. She was the first woman to win a Nobel Prize, the first 
person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. 
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever 
married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes. 
She was, in 1906, the first woman to become a professor at the University of Paris. """

documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [24]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person'), Node(id='Nobel Prize', type='Award'), Node(id='University Of Paris', type='Organization'), Node(id='Polish', type='Nationality'), Node(id='French', type='Nationality'), Node(id='Physicist', type='Profession'), Node(id='Chemist', type='Profession'), Node(id='Radioactivity', type='Research area'), Node(id='Professor', type='Profession')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish', type='Nationality'), type='HAS_NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='French', type='Nationality'), type='HAS_NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Physicist', type='Profession'), type='HAS_PROFESSION'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Chemist', type='Profession'), type='HAS_PROFESSION'), Relationship(source=Node(id='Marie Curie'

In [ ]:
NEO4J_URI=os.getenv('NEO4J_URI') 
NEO4J_USERNAME=os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD=os.getenv('NEO4J_PASSWORD')

knowledge_graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
knowledge_graph.add_graph_documents(graph_documents)

In [20]:
from pydantic import BaseModel, Field
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [14]:
QUERY_ENTITY_EXTRACT_PROMPT = ( "A question is provided below. Given the question, extract up to 5 " "entity names and types from the text. Focus on extracting the key entities " "that we can use to best lookup answers to the question. Avoid stopwords.\n" "---------------------\n" "{question}\n" "---------------------\n" "{format_instructions}\n" ) 
def extract_entities(llm): 
    # prompt = ChatPromptTemplate.from_messages([keyword_extraction_prompt]) 
    
    class SimpleNode(BaseModel): 
        """Represents a node in a graph with associated properties.""" 
        id: str = Field(description="Name or human-readable unique identifier.") 
        type: str = Field("Person", description="The type or label of the node.") 
    
    class SimpleNodeList(BaseModel): 
        """Represents a list of simple nodes.""" 
        nodes: List[SimpleNode] 
        
    output_parser = JsonOutputParser(pydantic_object=graph_documents[0].nodes)  
    return ( RunnablePassthrough.assign( format_instructions=lambda _: output_parser.get_format_instructions(), ) | ChatPromptTemplate.from_messages([QUERY_ENTITY_EXTRACT_PROMPT]) | llm | JsonOutputParser | RunnableLambda( lambda node_list: [(n["id"], n["type"]) for n in node_list["nodes"]]) )

In [16]:
from pydantic import BaseModel, Field
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define the prompt template
QUERY_ENTITY_EXTRACT_PROMPT = (
    "A question is provided below. Given the question, extract up to 5 "
    "entity names and types from the text. Focus on extracting the key entities "
    "that we can use to best lookup answers to the question. Avoid stopwords.\n"
    "---------------------\n"
    "{question}\n"
    "---------------------\n"
    "{format_instructions}\n"
)

# Define the data models
class SimpleNode(BaseModel):
    """Represents a node in a graph with associated properties."""
    id: str = Field(description="Name or human-readable unique identifier.")
    type: str = Field("Person", description="The type or label of the node.")

class SimpleNodeList(BaseModel):
    """Represents a list of simple nodes."""
    nodes: List[SimpleNode]

# Define the function to extract entities
def extract_entities(llm):
    # Create an output parser with the correct pydantic model
    output_parser = JsonOutputParser(pydantic_object=SimpleNodeList)

    return (
        RunnablePassthrough.assign(
            format_instructions=lambda _: output_parser.get_format_instructions(),
        )
        | ChatPromptTemplate.from_messages([QUERY_ENTITY_EXTRACT_PROMPT])
        | llm
        | output_parser
        | RunnableLambda(lambda node_list: [(n.id, n.type) for n in node_list.nodes])
    )


In [19]:
# Example showing extracted entities (nodes) 

extract_entities(graph_documents[0].nodes).invoke({ "question": "Who is Marie Curie?"}) # Output: [Marie Curie(Person)]

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [ ]:
def _combine_relations(relations): 
    return "\n".join(map(repr, relations)) 

ANSWER_PROMPT = ( "The original question is given below." "This question has been used to retrieve information from a knowledge graph." "The matching triples are shown below." "Use the information in the triples to answer the original question.\n\n" "Original Question: {question}\n\n" "Knowledge Graph Triples:\n{context}\n\n" "Response:" ) 
chain = ( { "question": RunnablePassthrough() } # extract_entities is provided by the Cassandra knowledge graph library
                                                # and extracts entitise as shown above.  
        | RunnablePassthrough.assign(entities = extract_entities(llm))  
        | RunnablePassthrough.assign( # graph_store.as_runnable() is provided by the CassandraGraphStore 
                                    # and takes one or more entities and retrieves the relevant sub-graph(s). 
            triples = itemgetter("entities") | graph_store.as_runnable()) | RunnablePassthrough.assign( context = itemgetter("triples") | RunnableLambda(_combine_relations)) | ChatPromptTemplate.from_messages([ANSWER_PROMPT]) | llm )

In [ ]:
chain.invoke("Who is Marie Curie?") 
# Output AIMessage( content="Marie Curie is a Polish and French chemist, physicist, and professor who " "researched radioactivity. She was married to Pierre Curie and has worked at " "the University of Paris. She is also a recipient of the Nobel Prize.", response_metadata={ 'token_usage': {'completion_tokens': 47, 'prompt_tokens': 213, 'total_tokens': 260}, 'model_name': 'gpt-4', ... } )

### 이건 그냥 예제 그 자체

In [26]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain.schema import Document
import openai

# LLM과 지식 그래프 변환기 초기화
llm = ChatOpenAI(temperature=0, model="gpt-4")
llm_transformer = LLMGraphTransformer(llm=llm)

# 문서 생성
text = """Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted 
pioneering research on radioactivity. She was the first woman to win a Nobel Prize, the first 
person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields. 
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever 
married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes. 
She was, in 1906, the first woman to become a professor at the University of Paris."""
documents = [Document(page_content=text)]

# 지식 그래프로 변환
graph_documents = llm_transformer.convert_to_graph_documents(documents)

# 지식 그래프를 바탕으로 Language Model에게 질문과 함께 전달
def ask_question_using_graph(llm, question, graph_documents):
    graph_info = []
    
    for graph_doc in graph_documents:
        # 노드와 관계를 텍스트로 변환
        for node in graph_doc.nodes:
            graph_info.append(f"Entity: {node['entity']} (Type: {node['type']})")
        
        for rel in graph_doc.relationships:
            graph_info.append(f"Relationship: {rel['from']} - {rel['relationship']} - {rel['to']}")
    
    # 그래프 정보를 결합
    graph_info_text = "\n".join(graph_info)
    
    # 프롬프트 생성
    prompt = f"The following is a knowledge graph based on the provided text:\n\n{graph_info_text}\n\nQuestion: {question}\nAnswer:"

    response = llm(prompt)
    return response.strip()

# 예시 질문
question = "What Nobel Prizes did Marie Curie win?"
answer = ask_question_using_graph(llm, question, graph_documents)

print(answer)


TypeError: 'Node' object is not subscriptable